# PROJET 1 : MYSQL DATA CLEANING

## Introduction

Ce premier projet porte sur le nettoyage de données d'un dataset téléchargé directement sur kaggle.com
Ce document traite sur les licenciements réalisées sur différentes entreprises axé dans le domaine de la technologie, et qui a connu un déclin d'emplois sur la période COVID19 à 2022.

### Etapes

Lien du dataset: https://www.kaggle.com/datasets/swaptr/layoffs-2022/data

#### Importation du fichier

La première étape de ce projet est d'importer ce dataset dans MYSQL.
Pour cela, on crée une database que l'on nomme par exemple "world_layoffs" puis on réalise une table que l'on nomme "layoffs" ou "licenciements". Sur cette table, on réalise un clic droit puis on clique sur "Table Data Import Wizard".
A ce moment là on peut donc choisir notre fichier au format CSV. 
On peut également choisir le format de chaque colonnes mais pour ce projet, on modifiera tout cela par la suite

#### Premier affichage

In [ ]:
SELECT * 
FROM layoffs;

#### Quelles étapes ?

Pour réaliser ce Data Cleaning on va se focaliser sur différentes étapes à suivre.
- Créer une seconde copie de la table en cas d'erreurs que l'on nomme "layoffs_test"
- Nous devons d'abord vérifier s'il n'y pas de duplicates ou doublons.
- Standardiser les données, pour éviter toutes erreurs par la suite.
- Supprimer les colonnes et lignes inutiles.


#### Supprimer les doublons

In [ ]:
# Création d'une seconde table layoffs_test "copie" de layoffs.

SELECT *
FROM layoffs;

CREATE TABLE world_layoffs.layoffs_test
LIKE world_layoffs.layoffs;

INSERT layoffs_test
SELECT * FROM world_layoffs.layoffs;

SELECT * 
FROM layoffs_test

In [ ]:
# Recherche de doublons/duplicates

SELECT *, COUNT(*) AS count
FROM world_layoffs.layoffs_test
GROUP BY company, location, industry, total_laid_off, percentage_laid_off, date, stage, country, funds_raised
HAVING COUNT(*) > 1;

# Ce code permet de recherche dans toute la table, chaque colonne identique et compter le nombre de doublons.
# Dans notre cas, nous trouvons un total de 2 doublons, 1 pour l'entreprise Cazoo et une autre pour l'entreprise Beyond Meat.

# Il ne faut pas oublier qu'une ligne est considérée comme doublon lorsque TOUTES ses lignes sont identiques.

In [ ]:
# Pour me repérer facilement, je crée une colonne id que j'incrémente de façon croissante.

ALTER TABLE world_layoffs.layoffs_test
ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY;

In [ ]:
# Je crée une table temporaire d'id pour éviter les erreurs
# Puis je supprime les ids liées aux doublons.

CREATE TABLE temp_ids (id INT);
INSERT INTO temp_ids (id)
SELECT MIN(id)
FROM world_layoffs.layoffs_test
GROUP BY company, location, industry, total_laid_off, percentage_laid_off, `date`, stage, country, funds_raised
HAVING COUNT(*) > 1;

DELETE FROM world_layoffs.layoffs_test
WHERE id IN (SELECT id FROM temp_ids);

In [ ]:
# Vérification des doublons des entreprises "Cazoo" et "Beyond Meat"

SELECT * 
FROM layoffs_test
WHERE company = "Cazoo";

SELECT * 
FROM layoffs_test
WHERE company = "Beyond Meat"; 

#Ces compagnies qui possédaient chacune 1 doublon n'en ont plus. Mais une ligne de "null" reste tout de même présente.

In [ ]:
# On supprime la colonne id qui nous servait de repère pour la suppression des doublons.
ALTER TABLE world_layoffs.layoffs_test
DROP COLUMN id;


In [ ]:
# On sait que le dataset possède obligatoirement un nom de compagnie, on se réfère à cette colonne pour la suppression de ligne null, et on rajoute
# les autres colonnes.

DELETE FROM world_layoffs.layoffs_test
WHERE company IS NULL 
  AND location IS NULL 
  AND industry IS NULL 
  AND total_laid_off IS NULL 
  AND percentage_laid_off IS NULL 
  AND `date` IS NULL 
  AND stage IS NULL 
  AND country IS NULL 
  AND funds_raised IS NULL;

# On vérifie à nouveau et c'est bon, les lignes "Null" sont supprimées.
SELECT * 
FROM layoffs_test
WHERE company = "Cazoo";

SELECT * 
FROM layoffs_test
WHERE company = "Beyond Meat";

#### Standardiser 

In [ ]:
# Je réalise une seconde copie de layoffs_test pour réaliser la standardisation.

# Je commence par un TRIM du nom des compagnies pour éviter tous espaces non visibles qui peuvent se placer avant ou après le nom.

SELECT *
FROM world_layoffs.layoffs_test2;

UPDATE world_layoffs.layoffs_test2
SET company = TRIM(company);

# Vérification compagnie : rien à signaler
SELECT DISTINCT company
FROM world_layoffs.layoffs_test2
ORDER BY company;

# Vérification industrie : une ligne non renseignée et une industrie renseigné en tant qu'URL.
SELECT DISTINCT industry
FROM world_layoffs.layoffs_test2
ORDER BY industry;

# Je vérifie s'il y a une ligne dont l'industrie commencent par https:

UPDATE world_layoffs.layoffs_test2
SET industry = 'N/A'
WHERE industry LIKE 'https:%';

SELECT *
FROM world_layoffs.layoffs_test2
WHERE company LIKE 'ebay';
# Changement en N/A pour l'industrie eBay pour ne pas avoir une URL.

# Vérification location : pas de problèmes, l'affiche est différent pour certains caractères mais pas de soucis pour ça
# exemple : Fayetteville, Düsseldorf

SELECT DISTINCT location
FROM world_layoffs.layoffs_test2
ORDER BY location;

# Vérification country : rien à signaler
SELECT DISTINCT country
FROM world_layoffs.layoffs_test2
ORDER BY country;



#### Colonnes et lignes inutiles type NULL / BLANK

In [ ]:
SELECT *
FROM layoffs_test2;

# Les colonnes/lignes vides et NULL se trouvent dans total_laid_off et percentage_laid_off.
# Je modifie ces colonnes à STRING vide pour les transformer en BLANK.

UPDATE world_layoffs.layoffs_test2
SET total_laid_off = NULL
WHERE total_laid_off = '';

UPDATE world_layoffs.layoffs_test2
SET percentage_laid_off = NULL
WHERE percentage_laid_off = '';

# Etant donné qu'on travaille sur des données de licenciement, si nous n'avons aucune informations sur ces deux colonnes alors elles nous sont inutiles.
# On doit donc les supprimer

DELETE FROM world_layoffs.layoffs_test2
WHERE total_laid_off IS NULL
AND percentage_laid_off IS NULL;

SELECT * 
FROM world_layoffs.layoffs_test2;

## Conclusion

Ainsi se conclut ce projet de Data Cleaning via MySQL. Les données sont désormais utilisables à l'analyse de données.
La question à se poser est : Quelles données nous seront utiles ?
Il faut donc supprimer les colonnes/lignes inutiles, vérifier qu'il n'y ai pas d'erreurs sur celles-ci et apporter les modification nécessaires.